## first simple domain adaptation (noise + lobe segments)

In [1]:
import numpy as np
import nibabel as nib
from scipy.ndimage import gaussian_filter
import glob
import shutil
import os
from tqdm import tqdm
import yaml
from multiprocessing import Pool

In [2]:
def add_noise(volume, mean, std):
    noise = np.random.normal(mean, std, volume.shape)
    noise = noise.reshape(volume.shape)
    
    num_salt = 512*512*500 * 0.0001
    coords = [np.random.randint(5, i - 5, int(num_salt)) for i in volume.shape]
    for x, y, z in zip(coords[0],coords[1],coords[2]):
        noise[x-1:x+2, y-1:y+2, z-1:z+2] += -0.8
        
    gauss = gaussian_filter(noise, sigma=1)
    noisy = volume + gauss
    
    return noisy, gauss

In [3]:
def adapt(args):
    in_dir, out_dir, orig_dir, trachea = args
    hash = in_dir.split('/')[-1]
    with open(f'{in_dir}/config.yml', "r") as filepath:
        config = yaml.safe_load(filepath)
    scan = config['Forest']['segment_root_path'][-13:-10]
    
    vol = nib.load(in_dir + '/volume.nii.gz').get_fdata().astype(np.float32)
    vol = vol / 255

    lung_mask = nib.load(in_dir + '/lung.nii.gz').get_fdata().astype(np.uint8)
    lung_mask = lung_mask >= 1

    wall_mask = nib.load(in_dir + '/airway_wall.nii.gz').get_fdata().astype(np.uint8)
    wall_mask = wall_mask >= 1

    airway_mask = nib.load(in_dir + '/airways.nii.gz').get_fdata().astype(np.uint8)
    airway_mask = airway_mask >= 1

    # add lung background
    adapted_vol = lung_mask.astype(np.uint8) * 0.9
    # add airway walls
    adapted_vol[wall_mask] = vol[wall_mask] / 10
    # add airways
    adapted_vol[airway_mask] = vol[airway_mask]
    adapted_vol = gaussian_filter(adapted_vol, sigma=0.8)

    del vol, wall_mask
    
    adapted_vol, _ = add_noise(adapted_vol, mean=0, std=0.2)
    
    adapted_vol *= -950

    orig_img = nib.load(f'{orig_dir}/images/ATM_{scan}_0000.nii.gz')
    orig_vol = orig_img.get_fdata()
    affine = orig_img.affine
    del orig_img

    # add original background
    adapted_vol[lung_mask == False] = orig_vol[lung_mask == False]
    adapted_vol = adapted_vol.astype(np.float32)

    nifti = nib.Nifti1Image(adapted_vol, affine)
    nib.save(nifti, f"{out_dir}/images/{hash}_adapted_vol.nii.gz")

    if trachea:
        orig_seg = nib.load(f'{orig_dir}/labels/ATM_{scan}_0000.nii.gz').get_fdata()
        main_bronchi = np.where(np.bitwise_and(lung_mask == False, orig_seg >= 1), True, False)
        adapted_airway = airway_mask.astype(np.uint8)
        adapted_airway[main_bronchi] = 1
        nifti = nib.Nifti1Image(adapted_airway, affine) #np.eye(4))
    else:
        nifti = nib.Nifti1Image(airway_mask.astype(np.uint8), affine)
    nib.save(nifti, f"{out_dir}/labels/{hash}_adapted_label.nii.gz")

## dataset_2

In [ ]:
in_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/dataset_1'
out_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/dataset_2'
    
for folder in glob.glob('/home/ahaas/airway-seg/vessel_graph_generation/datasets/lobes/*'):
    hash = folder.split('/')[-1]
    shutil.copyfile(f'{in_dir}/images/{hash}_adapted_vol.nii.gz', f'{out_dir}/images/{hash}_adapted_vol.nii.gz')
    affine = nib.load(f'{in_dir}/images/{hash}_adapted_vol.nii.gz').affine
    
    airway_mask = np.load(folder + '/airways.npy').astype(np.uint8)
    airway_mask = airway_mask >= 1
    
    nifti = nib.Nifti1Image(airway_mask.astype(np.uint8), affine) #np.eye(4))
    nib.save(nifti, f"{out_dir}/labels/{hash}_adapted_label.nii.gz")

## dataset_3

In [ ]:
in_dirs = sorted(glob.glob('/home/ahaas/airway-seg/vessel_graph_generation/datasets/lobes/20230817*'))
out_dir = '/home/ahaas/airway-seg/vessel_graph_generation/datasets/dataset_3'
print(out_dir)
orig_dir = '/home/shared/Data/ATM22/train/'
os.makedirs(f'{out_dir}/images', exist_ok=True)
os.makedirs(f'{out_dir}/labels', exist_ok=True)
args = []
for in_dir in in_dirs:
    args.append((in_dir, out_dir, orig_dir, False))
    #adapt(in_dir=in_dir, out_dir=out_dir, orig_dir=orig_dir, trachea=False)
with Pool(15) as p, tqdm(total=len(args)) as pbar:
    for _ in p.imap_unordered(adapt, args):
        pbar.update()